In [ ]:
DEVICES = "1"

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = DEVICES

import numpy as np
import pandas as pd
import cudf
import plotly.express as px
from tqdm import tqdm

tqdm.pandas()

# Import data

In [ ]:
ds = cudf.read_parquet("../data/gen/preprocessed_2023-08-28T11-09-39.parquet")

## We keep only the success loans

In [ ]:
success = ds.loanAmount == ds.loanFundraisingInfo_fundedAmount
counts = success.value_counts()
counts[True] / (counts[True] + counts[False]), len(ds)

In [ ]:
ds = ds[success]

In [ ]:
ds.dropna(subset=["raisedDate"], inplace=True)

In [ ]:
ds = ds.sample(1000)

# Number of Tags vs Time

In [ ]:
PT = ds.drop(columns=["lender_id", "lender_name", "lender_publicId", "loan_shareAmount", "loan_date", "loan_id"])
PT.drop_duplicates(inplace=True)

In [ ]:
time_df = PT[["fundraisingDate", "raisedDate"]].to_pandas()
time_df["date"] = time_df.progress_apply(
    lambda row: list(pd.period_range(row["fundraisingDate"], row["raisedDate"], freq="M")), axis=1
)

In [ ]:
time_df.info()

In [ ]:
time_df["date"] = time_df["date"].dt.to_timestamp()

In [ ]:
time_ds = cudf.from_pandas(time_df)
del time_df
time_ds.head()